### End of day system

Based on signal, the model will choose wether to buy a stock at open price, and sell it at close. 
The alternative is to do nothing, if insufficient signal.





In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import datetime

##  NEW

In [24]:

df = pd.read_csv('Stocks/abac.us.txt', index_col='Date')
df.index = pd.to_datetime(df.index)

# Add features

historySteps = 2 #10
df['DeltaV'] = df['Volume'] - df['Volume'].shift(1)
df['log_rtn'] = np.log(df['Close']).diff() # Log returns over day
df['in_log_rtn'] = np.log(df['Close'])-np.log(df['Open']) # Intraday log return

df['sigma_30'] = df['log_rtn'].rolling(window=30).std(ddof=1) * np.sqrt(252)
df['sigma_7'] = df['log_rtn'].rolling(window=7).std(ddof=1) * np.sqrt(252)

df['mu_30'] = df['log_rtn'].rolling(window=30).mean(ddof=1) * np.sqrt(252)
df['mu_7'] = df['log_rtn'].rolling(window=7).mean(ddof=1) * np.sqrt(252)

df['Open_z'] = (df['Open'] - df['mu_30'])/df['sigma_30']

# Add closing price of previous days
for i in range(1, historySteps):
    df[f'Close T-{i}'] = df['Close'].shift(i)

# Add open price of previous days
for i in range(1, historySteps):
    df[f'Open T-{i}'] = df['Open'].shift(i)

# Add high price of previous days
for i in range(1, historySteps):
    df[f'High T-{i}'] = df['High'].shift(i)

# Add low price of previous days
for i in range(1, historySteps):
    df[f'Low T-{i}'] = df['Low'].shift(i)

# Add volume of previous days
#for i in range(1, historySteps):
#    df[f'Volume T-{i}'] = df['Volume'].shift(i)

# Add volume change of previous days
#for i in range(1, historySteps):
#    df[f'DeltaV T-{i}'] = df['DeltaV'].shift(i)

# Add log return of previous days
for i in range(1, historySteps):
    df[f'log_rtn T-{i}'] = df['log_rtn'].shift(i)

# Add intraday log return of previous days
for i in range(1, historySteps):
    df[f'in_log_rtn T-{i}'] = df['in_log_rtn'].shift(i)

# Add volatility of previous days
for i in range(1, historySteps):
    df[f'sigma_30 T-{i}'] = df['sigma_30'].shift(i)
for i in range(1, historySteps):
    df[f'sigma_7 T-{i}'] = df['sigma_7'].shift(i)


# Add month and day
#df['Month'] = df.index.month
#df['Day'] = df.index.day
#df['Weekday'] = df.index.weekday
#df['Week'] = df.index.week



df = df.dropna()
df

/var/folders/0r/85hkrl9x0wq5b77x85v_hfxm0000gn/T/ipykernel_46792/3404672635.py:14: FutureWarning: Passing additional kwargs to Rolling.mean has no impact on the result and is deprecated. This will raise a TypeError in a future version of pandas.
  df['mu_30'] = df['log_rtn'].rolling(window=30).mean(ddof=1) * np.sqrt(252)
/var/folders/0r/85hkrl9x0wq5b77x85v_hfxm0000gn/T/ipykernel_46792/3404672635.py:15: FutureWarning: Passing additional kwargs to Rolling.mean has no impact on the result and is deprecated. This will raise a TypeError in a future version of pandas.
  df['mu_7'] = df['log_rtn'].rolling(window=7).mean(ddof=1) * np.sqrt(252)


,Open,High,Low,Close,Volume,OpenInt,DeltaV,log_rtn,in_log_rtn,sigma_30,...,mu_7,Open_z,Close T-1,Open T-1,High T-1,Low T-1,log_rtn T-1,in_log_rtn T-1,sigma_30 T-1,sigma_7 T-1
Date,,,,,,,,,,,,,,,,,,,,,
2010-08-31,15.6400,15.6400,14.9200,15.4000,2588,0,-875.0,0.026317,-0.015464,0.981184,...,-0.040862,16.117362,15.0000,15.2800,15.2800,14.84,-0.039221,-0.018495,1.018982,0.472520
2010-09-01,15.1200,15.8000,14.6000,15.2000,4745,0,2157.0,-0.013072,0.005277,0.729070,...,-0.029645,20.819968,15.4000,15.6400,15.6400,14.92,0.026317,-0.015464,0.981184,0.508198
2010-09-02,15.2000,15.4800,15.2000,15.2800,1288,0,-3457.0,0.005249,0.005249,0.729528,...,-0.029490,20.911050,15.2000,15.1200,15.8000,14.60,-0.013072,0.005277,0.729070,0.502722
2010-09-03,15.8800,15.8800,15.0000,15.0000,4819,0,3531.0,-0.018495,-0.057010,0.715728,...,0.030441,22.238615,15.2800,15.2000,15.4800,15.20,0.005249,0.005249,0.729528,0.502762
2010-09-07,15.8400,15.8800,15.0800,15.2400,2704,0,-2115.0,0.015873,-0.038615,0.717241,...,0.029958,22.116832,15.0000,15.8800,15.8800,15.00,-0.018495,-0.057010,0.715728,0.426984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-11-06,3.2900,3.4400,3.2900,3.4000,14528,0,11447.0,0.000000,0.032888,0.911851,...,0.000000,3.347460,3.4000,3.3001,3.4000,3.23,0.000000,0.029823,0.939824,0.539997
2017-11-07,3.3941,3.4400,3.2600,3.3540,30441,0,15913.0,-0.013622,-0.011885,0.915496,...,-0.118167,3.461018,3.4000,3.2900,3.4400,3.29,0.000000,0.032888,0.911851,0.532631
2017-11-08,3.3900,3.5815,3.3000,3.4747,15678,0,-14763.0,0.035354,0.024678,0.917070,...,0.116985,3.433099,3.3540,3.3941,3.4400,3.26,-0.013622,-0.011885,0.915496,0.461510


In [25]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer


from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor # Best
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler


import matplotlib.pyplot as plt




def predict(date, target='Close', verbose=False):
    df_train = df.loc[df.index < date]
    df_test = df.loc[df.index == date]
    y_train = df_train[target]
    X_train = df_train.drop(['Open', 'High', 'Low', 'Close', 'Volume', 
        'OpenInt', 'log_rtn', 'in_log_rtn', 'sigma_30', 'sigma_7', 'DeltaV'], axis=1)
    y_test = df_test[target]
    X_test = df_test.drop(['Open', 'High', 'Low', 'Close', 'Volume', 
        'OpenInt', 'log_rtn', 'in_log_rtn', 'sigma_30', 'sigma_7', 'DeltaV'], axis=1)
    
    
    model = Pipeline([
    ('normalizer', StandardScaler()),
    ('regressor', LinearRegression())
    ])

    model.fit(X_train, y_train)
    result = model.predict(X_test)

    # Get feature names
    feature_names = X_train.columns

    # Get coefficients
    coefs = model.named_steps['regressor'].coef_

    # Combine them nicely
    importances = sorted(zip(feature_names, coefs), key=lambda x: abs(x[1]), reverse=True)

    print("Feature importances:")
    for feature, coef in importances:
        print(f"{feature}: {coef:.4f}")
    

    if verbose:
        print(f"True value: {y_test.values[0]}")
        print(f"Predicted value: {round(result[0])}")
        print(f"Percentage missed: {round(100*np.abs((result[0]-y_test.values[0])/y_test.values[0]),2)}%")

        

    return result[0]

day = np.max(df.index - datetime.timedelta(2))



### Volume prediction

In [4]:
daysToPredict = 10

predictionList = []
for date in df.index[-daysToPredict:]:
    predictionList.append(predict(date, 'DeltaV'))



Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [5]:
prediction_df = pd.DataFrame({ 'Volume':df['Volume'][-daysToPredict:], 'DeltaV':df['DeltaV'][-daysToPredict:],
    'pred_DeltaV':predictionList})
prediction_df['pred_Volume'] = prediction_df['Volume'].shift(1) + prediction_df['pred_DeltaV']
prediction_df['dummy_Volume'] = prediction_df['Volume'].shift(1)
prediction_df['Model outperformance'] = np.abs(prediction_df['dummy_Volume'] - prediction_df['Volume']) - np.abs(prediction_df['pred_Volume'] - prediction_df['Volume'])

prediction_df

,Volume,DeltaV,pred_DeltaV,pred_Volume,dummy_Volume,Model outperformance
Date,,,,,,
2017-10-30,5254,-10339.0,-16024.857634,NaN,NaN,NaN
2017-10-31,1168,-4086.0,2421.024480,7675.024480,5254.0,-2421.024480
2017-11-01,10415,9247.0,7227.304851,8395.304851,1168.0,7227.304851
2017-11-02,5850,-4565.0,974.358179,11389.358179,10415.0,-974.358179
2017-11-03,3081,-2769.0,-4354.531478,1495.468522,5850.0,1183.468522
2017-11-06,14528,11447.0,18954.584654,22035.584654,3081.0,3939.415346
2017-11-07,30441,15913.0,-7591.563217,6936.436783,14528.0,-7591.563217
2017-11-08,15678,-14763.0,-23512.432290,6928.567710,30441.0,6013.567710
2017-11-09,28925,13247.0,-4767.823218,10910.176782,15678.0,-4767.823218


In [6]:
######### Delta V

fig = px.line(title='Predicting Change in Volume')

fig.add_scatter(x=prediction_df.index, y=prediction_df['DeltaV'], name='Reality')
fig.add_scatter(x=prediction_df.index, y=prediction_df['pred_DeltaV'], name='Prediction')

fig.update_layout(template='plotly_dark')
fig.show()


fig = px.line(title='Predicting Volume')
fig.add_scatter(x=prediction_df.index, y=prediction_df['Volume'], name='Reality')
fig.add_scatter(x=prediction_df.index, y=prediction_df['pred_Volume'], name='Prediction')

fig.add_scatter(x=prediction_df.index, y=prediction_df['dummy_Volume'], name='Dummy')
fig.add_scatter(x=prediction_df.index, y=prediction_df['Model outperformance'], name='Model Outperformance')


fig.add_hline(y=np.mean(prediction_df['Model outperformance']))

fig.update_layout(template='plotly_dark')
fig.show()

np.mean(prediction_df['Model outperformance'])


1133.9695522404609

## Predict Intraday Price Spread

In [26]:
daysToPredict = 365

predictionList = []
for date in df.index[-daysToPredict:]:
    predictionList.append(predict(date, 'in_log_rtn'))


prediction_df = pd.DataFrame({ 'in_log_rtn':df['in_log_rtn'][-daysToPredict:], 
    'pred_in_log_rtn':predictionList})
prediction_df['Model outperformance'] = np.abs(prediction_df['in_log_rtn'] - prediction_df['pred_in_log_rtn']) - np.abs(prediction_df['in_log_rtn'])

prediction_df

Feature importances:
High T-1: -0.0384
Close T-1: 0.0329
mu_7: 0.0160
Open_z: -0.0112
sigma_30 T-1: -0.0083
mu_30: 0.0057
Low T-1: 0.0051
log_rtn T-1: -0.0045
Open T-1: 0.0018
in_log_rtn T-1: -0.0011
sigma_7 T-1: 0.0000
Feature importances:
High T-1: -0.0385
Close T-1: 0.0338
mu_7: 0.0161
Open_z: -0.0111
sigma_30 T-1: -0.0082
mu_30: 0.0056
Low T-1: 0.0053
log_rtn T-1: -0.0045
in_log_rtn T-1: -0.0013
Open T-1: 0.0007
sigma_7 T-1: 0.0001
Feature importances:
High T-1: -0.0385
Close T-1: 0.0358
mu_7: 0.0161
Open_z: -0.0112
sigma_30 T-1: -0.0083
mu_30: 0.0057
log_rtn T-1: -0.0046
Low T-1: 0.0039
in_log_rtn T-1: -0.0014
Open T-1: 0.0002
sigma_7 T-1: 0.0001
Feature importances:
High T-1: -0.0393
Close T-1: 0.0370
mu_7: 0.0161
Open_z: -0.0112
sigma_30 T-1: -0.0082
mu_30: 0.0057
log_rtn T-1: -0.0047
Low T-1: 0.0041
in_log_rtn T-1: -0.0015
Open T-1: -0.0005
sigma_7 T-1: 0.0001
Feature importances:
High T-1: -0.0386
Close T-1: 0.0358
mu_7: 0.0161
Open_z: -0.0111
sigma_30 T-1: -0.0082
mu_30: 0.00

,in_log_rtn,pred_in_log_rtn,Model outperformance
Date,,,
2016-06-03,-0.087895,-0.004022,-0.004022
2016-06-06,0.107452,0.026727,-0.026727
2016-06-07,-0.033170,0.007969,0.007969
2016-06-08,-0.113210,0.021095,0.021095
2016-06-09,0.036219,0.028749,-0.028749
...,...,...,...
2017-11-06,0.032888,0.011566,-0.011566
2017-11-07,-0.011885,0.004839,0.004839
2017-11-08,0.024678,0.019561,-0.019561


In [31]:

fig = px.line(title='Predicting Intraday Log Returns')

fig.add_scatter(x=prediction_df.index, y=prediction_df['in_log_rtn'], name='Reality')
fig.add_scatter(x=prediction_df.index, y=prediction_df['pred_in_log_rtn'], name='Prediction')
fig.add_scatter(x=prediction_df.index, y=prediction_df['Model outperformance'], name='Outperformance')
fig.add_hline( y=np.mean(prediction_df['Model outperformance']))

fig.update_layout(template='plotly_dark')
fig.show()

np.mean(prediction_df['Model outperformance'])


0.000617908171899254

### Trading algorithm:
- When prediced intraday log returns are positive, buy for fixed dollar amount at Open price, sell at Close price at that day.
- Doing this for a year, what are the profits?

In [28]:
profits = 0
for date in prediction_df.index:
    if prediction_df['pred_in_log_rtn'][date] > 0:
        shares_bought = 1/df['Open'][date] # By investing $1
        prof = shares_bought * df['Close'][date] - 1
        profits += prof
print(profits)

2.1807601084297827


In [29]:

def tradingAlgorithm(filename, daysToPredict=365):
    ########################## Create Dataset ####################
    historySteps = 2
    df = pd.read_csv(filename, index_col='Date')
    df.index = pd.to_datetime(df.index)

    df['DeltaV'] = df['Volume'] - df['Volume'].shift(1)
    df['log_rtn'] = np.log(df['Close']).diff()
    df['in_log_rtn'] = np.log(df['Close'])-np.log(df['Open'])

    df['sigma_30'] = df['log_rtn'].rolling(window=30).std(ddof=1) * np.sqrt(252)
    df['sigma_7'] = df['log_rtn'].rolling(window=7).std(ddof=1) * np.sqrt(252)

    df['mu_30'] = df['log_rtn'].rolling(window=30).mean(ddof=1) * np.sqrt(252)
    df['mu_7'] = df['log_rtn'].rolling(window=7).mean(ddof=1) * np.sqrt(252)

    df['Open_z'] = (df['Open'] - df['mu_30'])/df['sigma_30']

    # Add closing price of previous days
    for i in range(1, historySteps):
        df[f'Close T-{i}'] = df['Close'].shift(i)

    # Add open price of previous days
    for i in range(1, historySteps):
        df[f'Open T-{i}'] = df['Open'].shift(i)

    # Add high price of previous days
    for i in range(1, historySteps):
        df[f'High T-{i}'] = df['High'].shift(i)

    # Add low price of previous days
    for i in range(1, historySteps):
        df[f'Low T-{i}'] = df['Low'].shift(i)

    # Add volume of previous days
    #for i in range(1, historySteps):
    #    df[f'Volume T-{i}'] = df['Volume'].shift(i)

    # Add volume change of previous days
    #for i in range(1, historySteps):
    #    df[f'DeltaV T-{i}'] = df['DeltaV'].shift(i)

    # Add log return of previous days
    for i in range(1, historySteps):
        df[f'log_rtn T-{i}'] = df['log_rtn'].shift(i)

    # Add intraday log return of previous days
    for i in range(1, historySteps):
        df[f'in_log_rtn T-{i}'] = df['in_log_rtn'].shift(i)

    # Add volatility of previous days
    for i in range(1, historySteps):
        df[f'sigma_30 T-{i}'] = df['sigma_30'].shift(i)
    for i in range(1, historySteps):
        df[f'sigma_7 T-{i}'] = df['sigma_7'].shift(i)


    # Add month and day
    #df['Month'] = df.index.month
    #df['Day'] = df.index.day
    #df['Weekday'] = df.index.weekday
    #df['Week'] = df.index.week

    df = df.dropna()


    def predict(date, target='Volume', verbose=False):
        df_train = df.loc[df.index < date]
        df_test = df.loc[df.index == date]
        y_train = df_train[target]
        X_train = df_train.drop(['Open', 'High', 'Low', 'Close', 'Volume', 
            'OpenInt', 'log_rtn', 'in_log_rtn', 'sigma_30', 'sigma_7', 'DeltaV'], axis=1)
        y_test = df_test[target]
        X_test = df_test.drop(['Open', 'High', 'Low', 'Close', 'Volume', 
            'OpenInt', 'log_rtn', 'in_log_rtn', 'sigma_30', 'sigma_7', 'DeltaV'], axis=1)
        
        model = Pipeline([
        ('normalizer', StandardScaler()),
        ('regressor', LinearRegression())
        ])

        model.fit(X_train, y_train)
        result = model.predict(X_test)


        if verbose:
            print(f"True value: {y_test.values[0]}")
            print(f"Predicted value: {round(result[0])}")
            print(f"Percentage missed: {round(100*np.abs((result[0]-y_test.values[0])/y_test.values[0]),2)}%")

        return result[0]
    

    predictionList = []
    for date in df.index[-daysToPredict:]:
        predictionList.append(predict(date, 'in_log_rtn'))


    prediction_df = pd.DataFrame({ 'in_log_rtn':df['in_log_rtn'][-daysToPredict:], 
        'pred_in_log_rtn':predictionList})
    prediction_df['Model outperformance'] = np.abs(prediction_df['in_log_rtn'] - prediction_df['pred_in_log_rtn']) - np.abs(prediction_df['in_log_rtn'])

    profits = 0
    for date in prediction_df.index:
        if prediction_df['pred_in_log_rtn'][date] > 0:
            shares_bought = 1/df['Open'][date] # By investing $1
            prof = shares_bought * df['Close'][date] - 1
            profits += prof
    print(filename, profits)
    return profits 



In [30]:
tradingAlgorithm('Stocks/a.us.txt')

/var/folders/0r/85hkrl9x0wq5b77x85v_hfxm0000gn/T/ipykernel_46792/3657722851.py:14: FutureWarning:

Passing additional kwargs to Rolling.mean has no impact on the result and is deprecated. This will raise a TypeError in a future version of pandas.

/var/folders/0r/85hkrl9x0wq5b77x85v_hfxm0000gn/T/ipykernel_46792/3657722851.py:15: FutureWarning:

Passing additional kwargs to Rolling.mean has no impact on the result and is deprecated. This will raise a TypeError in a future version of pandas.



Stocks/a.us.txt 0.4712094510651569


0.4712094510651569

In [12]:
import os

import warnings
warnings.filterwarnings("ignore")

profitsList = []
directory = os.fsencode('Stocks')
        
for file in os.listdir(directory):
    if len(profitsList) > 50:
        break
    try:
        filename = os.fsdecode(file)
        if filename.endswith(".txt"): 
            profitsList.append(tradingAlgorithm('Stocks/'+filename))
    except:
        pass
  

Stocks/iba.us.txt 0.20459982342888205
Stocks/wpz.us.txt 0.18946908046244615
Stocks/flic.us.txt 0.15634586359834735
Stocks/nbhc.us.txt 0.12280762758638364
Stocks/ncs.us.txt -0.1114817445851054
Stocks/grvy.us.txt 2.7437773437178707
Stocks/lgcyo.us.txt 1.432113137991652
Stocks/eqco.us.txt 0.2245451441255708
Stocks/gpk.us.txt 0.026139184324132536
Stocks/wti.us.txt -0.24328661966691167
Stocks/dyn_a.us.txt -0.8158795004428342
Stocks/mygn.us.txt -0.06272956983032585
Stocks/cor_a-cl.us.txt 0
Stocks/jmu.us.txt -0.4818443415540292
Stocks/jpem.us.txt 0.02446272282407569
Stocks/kelya.us.txt 0.010293604398732725
Stocks/wbig.us.txt 0.1433263008430571
Stocks/nycb.us.txt -0.03674965515892259
Stocks/elp.us.txt 0.1013669143408128
Stocks/aieq.us.txt 0
Stocks/axas.us.txt -0.01702535068224864
Stocks/cphi.us.txt 1.2169163594798964
Stocks/thr.us.txt 0.027200551505044523
Stocks/nxeow.us.txt 5.247711154240214
Stocks/vrsk.us.txt 0.12160136271467259
Stocks/abil.us.txt -1.0240343661096176
Stocks/evep.us.txt 0.137

In [18]:
print("Annual return on 1$:", np.mean(profitsList))

print("Transaction costs:", 1*365*2*0.049*0.01)


Annual return on 1$: 0.17631106702864774
Transaction costs: 0.3577


In [13]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Histogram(x=profitsList)])
fig.show()
